# 天池二手车交易价格预测
## 读取数据


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from datetime import datetime
import numpy as np

In [2]:
# 数据加载
train_data = pd.read_csv('./used_car_train_20200313.csv', sep=' ')

In [3]:
test_data = pd.read_csv("./used_car_testB_20200421.csv", sep = ' ')

## 缺失值补全

In [4]:
train_data['notRepairedDamage'].value_counts()

0.0    111361
-       24324
1.0     14315
Name: notRepairedDamage, dtype: int64

In [5]:
test_data['notRepairedDamage'].value_counts()

0.0    37224
-       8069
1.0     4707
Name: notRepairedDamage, dtype: int64

In [6]:
train_data['notRepairedDamage'].replace('-', '0.0', inplace=True)
test_data['notRepairedDamage'].replace('-', '0.0', inplace=True)

In [7]:
# 查看数值类型
#train_data.info()
numerical_cols = train_data.select_dtypes(exclude='object').columns
numerical_cols

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'regionCode', 'seller', 'offerType',
       'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14'],
      dtype='object')

In [8]:
# 查看分类类型
categorical_cols = train_data.select_dtypes(include='object').columns
categorical_cols

Index(['notRepairedDamage'], dtype='object')

## 特征工程
### 特征选择

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
# 特征选择
# drop_cols = ['SaleID', 'regDate', 'creatDate', 'offerType', 'price']
drop_cols = ['SaleID','name','price']
feature_cols = [col for col in train_data.columns if col not in drop_cols]

### 日期类型处理

In [11]:
train_data['regDate'] = pd.to_datetime(train_data['regDate'], format='%Y%m%d', errors='coerce')
test_data['regDate']=pd.to_datetime(test_data['regDate'], format='%Y%m%d', errors='coerce')
train_data['regDate']

0        2004-04-02
1        2003-03-01
2        2004-04-03
3        1996-09-08
4        2012-01-03
            ...    
149995   2000-06-07
149996   2009-11-02
149997   2010-10-03
149998   2006-03-12
149999   1999-02-04
Name: regDate, Length: 150000, dtype: datetime64[ns]

In [12]:
train_data['regDate'].fillna(train_data['regDate'].mode()[0], inplace=True)
test_data['regDate'].fillna(test_data['regDate'].mode()[0], inplace=True)

In [13]:
train_data['regDate'] = train_data['regDate'].apply(lambda x: x.strftime('%Y%m%d'))
test_data['regDate'] = test_data['regDate'].apply(lambda x: x.strftime('%Y%m%d'))

### 添加年月日信息

In [15]:
train_data['regDate_year'] = train_data['regDate'].astype('str').apply(lambda x: int(x[0:4]))
train_data['regDate_month'] = train_data['regDate'].astype('str').apply(lambda x: int(x[4:6]))
train_data['regDate_day'] = train_data['regDate'].astype('str').apply(lambda x: int(x[6:]))

In [16]:
train_data['creatDate_year'] = train_data['creatDate'].astype('str').apply(lambda x: int(x[0:4]))
train_data['creatDate_month'] =train_data['creatDate'].astype('str').apply(lambda x: int(x[4:6]))
train_data['creatDate_day'] =train_data['creatDate'].astype('str').apply(lambda x: int(x[6:]))

In [17]:
test_data['regDate_year'] = test_data['regDate'].astype('str').apply(lambda x: int(x[0:4]))
test_data['regDate_month'] = test_data['regDate'].astype('str').apply(lambda x: int(x[4:6]))
test_data['regDate_day'] = test_data['regDate'].astype('str').apply(lambda x: int(x[6:]))

In [18]:
test_data['creatDate_year'] = test_data['creatDate'].astype('str').apply(lambda x: int(x[0:4]))
test_data['creatDate_month'] =test_data['creatDate'].astype('str').apply(lambda x: int(x[4:6]))
test_data['creatDate_day'] =test_data['creatDate'].astype('str').apply(lambda x: int(x[6:]))

In [19]:
train_data['regDate'] = train_data['regDate'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))
test_data['regDate'] = test_data['regDate'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))

In [20]:
# 时间diff
train_data['regDate_diff'] = (train_data['regDate'] - train_data['regDate'].min()).dt.days
test_data['regDate_diff'] = (test_data['regDate'] - test_data['regDate'].min()).dt.days
train_data[['regDate','regDate_diff']]

,regDate,regDate_diff
0,2004-04-02,4840
1,2003-03-01,4442
2,2004-04-03,4841
3,1996-09-08,2077
4,2012-01-03,7672
...,...,...
149995,2000-06-07,3445
149996,2009-11-02,6880
149997,2010-10-03,7215
149998,2006-03-12,5549


In [21]:
feature_cols.append('regDate_year')
feature_cols.append('creatDate_year')

In [22]:
# 提取特征列
X_data = train_data[feature_cols]
Y_data = train_data['price']
X_test = test_data[feature_cols]

In [23]:
# 定一个统计函数，用于统计某字段的特征
def show_stats(data):
    print('min: ', np.min(data))
    print('max: ', np.max(data))
    # ptp = max - min
    print('ptp: ', np.ptp(data))
    print('mean: ', np.mean(data))
    print('std: ', np.std(data))
    print('var: ', np.var(data))
show_stats(Y_data)

min:  11
max:  99999
ptp:  99988
mean:  5923.327333333334
std:  7501.973469876635
var:  56279605.942732885


In [24]:
#X_data.info()
X_data['notRepairedDamage'] = X_data['notRepairedDamage'].astype('float64')
X_test['notRepairedDamage'] = X_test['notRepairedDamage'].astype('float64')

### 添加品牌信息

In [25]:
# 处理品牌字段
brand_data = train_data.groupby('brand')
all_info = {}
for brand_index, brand_temp in brand_data:
    info = {}
    brand_temp = brand_temp[brand_temp['price'] > 0]
    info['brand_amount'] = len(brand_temp)
    info['brand_price_max'] = brand_temp.price.max()
    info['brand_price_min'] = brand_temp.price.min()
    info['brand_price_median'] = brand_temp.price.median()
    info['brand_price_mean'] = brand_temp.price.mean()
    info['brand_price_std'] = brand_temp.price.std()
    info['brand_price_ptp'] = info['brand_price_max'] - info['brand_price_min']
    all_info[brand_index] = info

In [26]:
df_brand = pd.DataFrame(all_info).T
df_brand = df_brand.reset_index()
df_brand.rename(columns={'index':'brand'}, inplace=True)
df_brand

,brand,brand_amount,brand_price_max,brand_price_min,brand_price_median,brand_price_mean,brand_price_std,brand_price_ptp
0,0,31480.0,68500.0,13.0,3199.0,5535.364994,6272.522119,68487.0
1,1,13794.0,99900.0,15.0,6499.0,9273.311947,9369.631497,99885.0
2,2,321.0,59800.0,35.0,7700.0,12037.822430,10875.892891,59765.0
3,3,2461.0,37500.0,65.0,4990.0,6482.822430,5396.327503,37435.0
4,4,16737.0,99999.0,12.0,6000.0,8538.788074,8472.393191,99987.0
5,5,4665.0,31500.0,20.0,2300.0,3306.349411,3343.624586,31480.0
6,6,10217.0,59900.0,13.0,1800.0,3611.840266,4681.293524,59887.0
7,7,2361.0,38900.0,60.0,2600.0,4196.064803,4752.032776,38840.0
8,8,2077.0,99999.0,30.0,2299.0,4900.515166,6227.035419,99969.0
9,9,7306.0,68530.0,50.0,1400.0,2438.274843,2974.272776,68480.0


In [27]:
# brand_stats = pd.DataFrame(all_info).T.reset_index().rename(columns={'index':'brand'})
train_data = train_data.merge(df_brand, how='left', on='brand')
test_data = test_data.merge(df_brand, how='left', on='brand')

In [28]:
print(feature_cols)

['regDate', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller', 'offerType', 'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'regDate_year', 'creatDate_year']


In [29]:
cols = train_data.columns.tolist()
cols.remove('creatDate')
cols.remove('SaleID')
cols.remove('regDate')
cols.remove('price')
cols.remove('name')
print(cols)

['model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode', 'seller', 'offerType', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'creatDate_year', 'regDate_year', 'regDate_month', 'regDate_day', 'creatDate_month', 'creatDate_day', 'regDate_diff', 'brand_amount', 'brand_price_max', 'brand_price_min', 'brand_price_median', 'brand_price_mean', 'brand_price_std', 'brand_price_ptp']


## 构造模型

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

In [31]:
X_data = train_data[cols]
Y_data = train_data['price']
X_test = test_data[cols]

In [32]:
X_data['notRepairedDamage'] = X_data['notRepairedDamage'].astype('float64')
X_test['notRepairedDamage'] = X_test['notRepairedDamage'].astype('float64')

In [33]:
# 数据集切分
x_train, x_val, y_train, y_val = train_test_split(X_data, Y_data, test_size=0.2)

**xgboost**

In [36]:
# 创建模型1
model = xgb.XGBRegressor(n_estimators=150, learning_rate=0.1, gamma=0,
                         subsample=0.8, colsample_bytree=0.8, max_depth=7,
                         random_state=2021)

In [37]:
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
mae = mean_absolute_error(y_val, y_pred)
print('xgb mae:', mae)

xgb mae: 612.2227533308665


In [34]:
# xgb 组合参数
model = xgb.XGBRegressor(max_depth=6, learning_rate=0.05, n_estimators=2000,
                         objective='reg:linear', tree_method='gpu_hist',
                         subsample=0.8, colsample_bytree=0.8,
                         min_child_samples=3, eval_metric='auc', reg_lambda=0.5)

In [35]:
model.fit(X_data, Y_data)

[23:48:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:48:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='auc',
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=6, min_child_samples=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=2000, n_jobs=4,
             num_parallel_tree=1, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=0.5, scale_pos_weight=1, subsample=0.8,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [36]:
y_pred = model.predict(X_test[cols])

In [37]:
# 训练 欠拟合 => n_estimators太小，或者 learning_rate太小
show_stats(y_pred)

min:  -1264.7198
max:  91061.734
ptp:  92326.45
mean:  5950.64
std:  7423.3623
var:  55106308.0


In [39]:
# 因为XGBoost是集成学习，多棵树组成
# 有些树的叶子节点 有可能为负
result = pd.DataFrame()
result['SaleID'] = test_data['SaleID']
result['price'] = y_pred
result[result['price'] < 0] = 11
result

,SaleID,price
0,200000,1298.507935
1,200001,1826.747192
2,200002,8398.540039
3,200003,1223.487061
4,200004,2086.377930
...,...,...
49995,249995,6683.768555
49996,249996,18572.890625
49997,249997,5939.886230
49998,249998,5189.446777


In [40]:
submit = pd.read_csv("./used_car_sample_submit.csv", sep = ',')

In [41]:
result['SaleID'] = submit['SaleID']

In [42]:
result.to_csv('./submission/baseline_xgb_20210424_2.csv', index=False)

**xgb的五折交叉验证**

In [43]:
from sklearn.model_selection import StratifiedKFold

In [44]:
def ensemble_model(clf, train_x, train_y, test):
    # 做五折交叉验证
    sk = StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)
    result= []
    mean_mae = 0
    for k, (train_index, val_index) in enumerate(sk.split(train_x, train_y)):
        train_x_real = train_x.iloc[train_index]
        train_y_real = train_y.iloc[train_index]
        val_x = train_x.iloc[val_index]
        val_y = train_y.iloc[val_index]
        
        clf = clf.fit(train_x_real, train_y_real)
        val_y_pred = clf.predict(val_x)
        
        mae_val = mean_absolute_error(val_y, val_y_pred)
        print('第{}个子模型MAE:{}'.format(k+1, mae_val))
        mean_mae += mae_val / 5
        # 子模型预测测试集
        test_y_pred = clf.predict(test)
        result.append(test_y_pred)
    print(mean_mae)
    mean_result = sum(result) / 5
    return mean_result

In [45]:
model_xgb = xgb.XGBRegressor(max_depth=6, learning_rate=0.05, n_estimators=2000,
                         objective='reg:linear', tree_method='gpu_hist',
                         subsample=0.8, colsample_bytree=0.8,
                         min_child_samples=3, eval_metric='auc', reg_lambda=0.5)

In [46]:
y_pred_kf5 = ensemble_model(model_xgb, X_data, Y_data, X_test[cols])

[23:51:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:51:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第1个子模型MAE:528.6467209724386
[23:52:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[23:52:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in 

In [47]:
result = pd.DataFrame()
result['SaleID'] = test_data['SaleID']
result['price'] = y_pred_kf5
result[result['price'] < 0] = 11
result

,SaleID,price
0,200000,1283.145874
1,200001,1870.215210
2,200002,8528.859375
3,200003,1173.996460
4,200004,2019.751953
...,...,...
49995,249995,6781.395996
49996,249996,18528.486328
49997,249997,6014.521973
49998,249998,4973.518555


In [48]:
result['SaleID'] = submit['SaleID']

In [49]:
result.to_csv('./submission/baseline_xgb_20210425_kf5.csv', index=False)